In [7]:
import pandas as pd
import time
import sys

exact_author='D.Restrepo.1'
UPDATE=False

old=pd.read_json('data/old.json')#.reset_index(drop=True)
#old.number_of_citations.sum()

# Download citations of each paper

if UPDATE:
    for r in old['recid']:
        p=pd.read_json('http://inspirehep.net/search?ln=en&p=refersto:recid:{}&of=recjson'.format(r)
                      ).to_json('data/{}.json'.format(r))
        print(r)
        time.sleep(1)

# Download INSPIRE HEP Profile

# TODO: Check if the author has more than 250 records!

url='http://inspirehep.net/search?p=ea+{}&rg=250&of=recjson'.format(exact_author)
df=pd.read_json(url)
new=df.copy()
#df.number_of_citations.sum()

ANALYSIS=False
if new['number_of_citations'].sum()>old['number_of_citations'].sum():
    ANALYSIS=True
if not ANALYSIS:
    sys.exit('NO NEW CITES TO ANALYSE!')

old['iid']=old.FIXME_OAI.apply(lambda d: d['id']).str.split(':').str[-1]
new['iid']=old.FIXME_OAI.apply(lambda d: d['id']).str.split(':').str[-1]

old=old.rename({'number_of_citations':'nc'},axis='columns') ##add other relevant columns for analysis

new=new.merge(old[['nc','iid']],on='iid',how='left')#[['number_of_citations','nc']]

news=new[new['number_of_citations']>new['nc']]

print('New citations in:')
news['recid']
#['primary_report_number'].apply(lambda l:  [x for x in l if x.find('arXiv')>-1])

for r in news['recid']:
    #Analysis of each citation
    print('===== NEW CITATION FOR {}====='.format(r))
    oldrefersto=pd.read_json('data/{}.json'.format(r))
    refersto=pd.read_json('http://inspirehep.net/search?ln=en&p=refersto:recid:{}&of=recjson'.format(r))
    print( news[news['recid']==r]['title'].values[0] )
    print('with')
    print(news[news['recid']==r]['authors'].apply(lambda l: [d['full_name'] for d in l]).values[0])
    print('WAS CITED BY:')
    try:
        newrecid=pd.np.setdiff1d(refersto['recid'],oldrefersto['recid'])
    except KeyError:
        newrecid=pd.np.setdiff1d(refersto['recid'],[])
    #newrecid=pd.np.setdiff1d(refersto['recid'],oldrefersto['recid'][:-2])
    for rr in newrecid:
        print('*'*60)
        print( refersto[refersto['recid']==rr]['title'].values[0])
        print( refersto[refersto['recid']==rr]['authors'].apply(lambda l: 
                                            [d['full_name'] for d in l]).values[0])
        try:
            doctype=refersto[refersto['recid']==rr]['primary_report_number'].apply(lambda l: 
                                            [x for x in l if x.find('arXiv')>-1]
                                            if type(l)==list else 
                                            [x for x in [l] if x.find('arXiv')>-1 ]).values[0]
            if not doctype: #force TypeError
                doctypetmp=refersto['forceKeyError']
        except TypeError:
            l=refersto[refersto['recid']==rr].get('collection')
            if not l.empty:
                if 'Thesis' in [d.get('primary') for d in l.values[0]]:
                    doctype='Thesis'
                else:
                    doctype='''WARNING: unkown doctype. Check: refersto[refersto['recid']==rr].get('collection')'''
            else:
                doctype='''WARNING: unkown doctype. Check: refersto[refersto['recid']==rr]'''                    
            
        print( doctype )    
    print('UPDATE CITATIONS (Y)')
    YN=input('Y/N')
    if YN not in ['N','n','not','Not']:
        print('UPDATING JSON DATA CITATION for {}'.format(r))
        refersto.to_json('data/{}.json'.format(r))

New citations in:
===== NEW CITATION FOR 1708907=====
{'title': 'Minimal radiative Dirac neutrino mass models'}
with
['Calle, Julian', 'Restrepo, Diego', 'Yaguna, Carlos E.', 'Zapata, Óscar']
WAS CITED BY:
************************************************************
{'title': 'Sequentially loop supressed fermion masses from a unique discrete symmetry'}
['Arbeláez, Carolina', 'Cárcamo Hernández, A.E.', 'Cepedello, Ricardo', 'Kovalenko, Sergey', 'Schmidt, Ivan']
['arXiv:1911.02033']
UPDATE CITATIONS (Y)
Y/Ny
UPDATING JSON DATA CITATION for 1708907
===== NEW CITATION FOR 1686334=====
{'title': 'Two-component dark matter and a massless neutrino in a new $B-L$ model'}
with
['Bernal, Nicolás', 'Restrepo, Diego', 'Yaguna, Carlos', 'Zapata, Óscar']
WAS CITED BY:
************************************************************
{'title': 'Sequentially loop supressed fermion masses from a unique discrete symmetry'}
['Arbeláez, Carolina', 'Cárcamo Hernández, A.E.', 'Cepedello, Ricardo', 'Kovalenko, Se

In [8]:
print('UPDATE FULL DATABASE for profiles (N)')
YN=input('Y/N')
if YN=='Y' or YN=='y' or YN=='yes' or YN=='Yes':
    df.to_json('data/old.json')

UPDATE FULL DATABASE for profiles (N)
Y/Ny
